# 2. Feature Pipeline

## 2.1. Setup

In [1]:
# Standard imports
import os
import sys
import json
import time
from datetime import date, timedelta
import warnings
from pathlib import Path
warnings.filterwarnings("ignore", module="IPython")

#  Establish project root directory
def find_project_root(start: Path):
    for parent in [start] + list(start.parents):
        if (parent / "pyproject.toml").exists():
            return parent
    return start

root_dir = find_project_root(Path().absolute())
print("Project root dir:", root_dir)

if str(root_dir) not in sys.path:
    sys.path.append(str(root_dir))

# Third-party imports
import requests
import pandas as pd
import great_expectations as gx
import hopsworks

#  Project imports
from utils import cleaning, config, feature_engineering, fetchers, hopsworks_admin, incremental, metadata

#  Load settings 
settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")
HOPSWORKS_API_KEY = settings.HOPSWORKS_API_KEY.get_secret_value()
GITHUB_USERNAME = settings.GH_USERNAME.get_secret_value()

# Login to Hopsworks
project = hopsworks.login(api_key_value=HOPSWORKS_API_KEY)
fs = project.get_feature_store()

Project root dir: c:\Users\krist\Documents\GitHub\pm25-forecast-openmeteo-aqicn
HopsworksSettings initialized!
2026-01-09 10:58:27,827 INFO: Initializing external client
2026-01-09 10:58:27,827 INFO: Base URL: https://c.app.hopsworks.ai:443
2026-01-09 10:58:28,667 WARNING: UserWarning: The installed hopsworks client version 4.1.2 may not be compatible with the connected Hopsworks backend version 4.2.2. 
To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'



2026-01-09 10:58:29,746 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1279184


Repository management

In [2]:
repo_dir = hopsworks_admin.clone_or_update_repo(GITHUB_USERNAME)
os.chdir(repo_dir)

Already in repo at c:\Users\krist\Documents\GitHub\pm25-forecast-openmeteo-aqicn


In [3]:
today = date.today()

if settings.AQICN_API_KEY is None:
    print("AQICN_API_KEY missing.")
    sys.exit(1)

AQICN_API_KEY = settings.AQICN_API_KEY.get_secret_value()

secrets = hopsworks.get_secrets_api()
try:
    secret = secrets.get_secret("AQICN_API_KEY")
    if secret is not None:
        secret.delete()
except Exception:
    pass

secrets.create_secret("AQICN_API_KEY", AQICN_API_KEY)

Secret created successfully, explore it at https://c.app.hopsworks.ai:443/account/secrets


Secret('AQICN_API_KEY', 'PRIVATE')

## 2.3. Get Feature Groups

In [4]:
air_quality_fg, sensor_metadata_fg, weather_fg = hopsworks_admin.create_feature_groups(fs)

## 2.4. Sensor Location Loading

In [5]:
# Load metadata from feature group
metadata_df = sensor_metadata_fg.read()
if len(metadata_df) == 0:
    print("⚠️ No sensor metadata found. Run pipeline 1 (backfill) first.")
    sys.exit(1)

print(f"📍 Loaded metadata for {len(metadata_df)} sensors")
metadata_df = metadata_df.set_index("sensor_id")

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.57s) 
📍 Loaded metadata for 103 sensors


In [6]:
# # Convert @ URLs to A URLs for Swedish sensors (AQICN API change)
# fixed_count = 0
# for sensor_id, location in locations.items():
#     if "@" in location["aqicn_url"]:
#         old_url = location["aqicn_url"]
#         new_url = old_url.replace("/@", "/A")
#         location["aqicn_url"] = new_url
#         fixed_count += 1

# if fixed_count > 0:
#     print(f"🔧 Fixed {fixed_count} sensor URLs from @ to A format")
# else:
#     print("ℹ️ All sensor URLs already in correct format")

## 2.5. Data Collection
Loop through all sensors to fetch today's air quality data and weather forecasts, format data to match feature group schemas.

In [ ]:
# Load metadata from feature group for nearby sensor calculations
metadata_indexed = metadata_df.copy()
# Ensure index is int type to match sensor_id values in data
metadata_indexed.index = metadata_indexed.index.astype(int)

successful_sensors = 0
failed_sensors = 0
skipped_sensors = 0
location_weather_uploaded = set()  # Track which location weather we've already uploaded

print(f"🔍 Processing {len(metadata_df)} sensor locations...")

# Get historical data once for all sensors (for rolling/lag features)
historical_start = today - timedelta(days=4)
try:
    # Read all data (Python env doesn't support column selection)
    historical_df = air_quality_fg.read()
    if not historical_df.empty:
        historical_df["date"] = pd.to_datetime(historical_df["date"]).dt.tz_localize(None)
        # Convert today to datetime for comparison
        today_dt = pd.to_datetime(today)
        historical_start_dt = pd.to_datetime(historical_start)
        # Filter in pandas instead
        historical_df = historical_df[
            (historical_df["date"] >= historical_start_dt) & (historical_df["date"] < today_dt)
        ][["date", "sensor_id", "pm25"]]
        # IMPORTANT: Only keep historical data for sensors that exist in metadata
        existing_sensor_ids = metadata_indexed.index.tolist()
        historical_df = historical_df[historical_df["sensor_id"].isin(existing_sensor_ids)]
    else:
        historical_df = pd.DataFrame()
except Exception as e:
    print(f"⚠️ Error reading historical data: {e}")
    historical_df = pd.DataFrame()

for sensor_id, meta in metadata_df.iterrows():
    max_retries = 3
    
    for attempt in range(max_retries):
        try:
            # Fetch current air quality
            aq_today_df = fetchers.get_pm25(meta["aqicn_url"], meta["country"], meta["city"], 
                                           meta["street"], today, AQICN_API_KEY)
            
            # Check if we got data
            if aq_today_df.empty or aq_today_df['pm25'].isna().all():
                print(f"⏭️ Sensor {sensor_id}: No AQ data available")
                skipped_sensors += 1
                break
            
            # Format air quality data
            aq_today_df = aq_today_df.assign(
                sensor_id=int(sensor_id),
                location_id=int(meta["location_id"])
            )
            aq_today_df["pm25"] = pd.to_numeric(aq_today_df["pm25"], errors="coerce").astype("float64")
            aq_today_df["date"] = pd.to_datetime(aq_today_df["date"]).dt.tz_localize(None)
            aq_today_df = aq_today_df.drop(columns=["url", "country", "city", "street"], errors="ignore")
            
            # Combine with historical data for this sensor
            sensor_historical = historical_df[historical_df["sensor_id"] == sensor_id] if not historical_df.empty else pd.DataFrame()
            combined = pd.concat([sensor_historical, aq_today_df], ignore_index=True) if not sensor_historical.empty else aq_today_df
            
            # Sort by date for proper lag/rolling calculations
            combined = combined.sort_values("date").reset_index(drop=True)
            
            # Add features
            combined = feature_engineering.add_rolling_window_feature(combined, window_days=3, column="pm25", new_column="pm25_rolling_3d")
            combined = feature_engineering.add_lagged_features(combined, column="pm25", lags=[1, 2, 3])
            
            # Add nearby sensor feature - only for this sensor's data
            combined = feature_engineering.add_nearby_sensor_feature(combined, metadata_indexed, n_closest=3)
            
            # Keep only today's data
            today_dt = pd.to_datetime(today)
            aq_final = combined[combined["date"].dt.date == today].copy()
            aq_final = aq_final.dropna(subset=['pm25'])
            
            if aq_final.empty:
                print(f"⏭️ Sensor {sensor_id}: No valid data after processing")
                skipped_sensors += 1
                break
            
            # Ensure correct data types and column order
            aq_final["sensor_id"] = aq_final["sensor_id"].astype("int32")
            aq_final["location_id"] = aq_final["location_id"].astype("int32")
            
            # Get expected columns from feature group
            fg_columns = [f.name for f in air_quality_fg.features]
            aq_final = aq_final[fg_columns]
            
            # Insert air quality data immediately
            air_quality_fg.insert(aq_final)
            
            # Fetch and upload weather for this location (if not already done)
            location_id = int(meta["location_id"])
            if location_id not in location_weather_uploaded:
                end_date = today + timedelta(days=7)
                weather_df = fetchers.get_weather_forecast(location_id, today, end_date, 
                                                          meta["latitude"], meta["longitude"])
                
                if not weather_df.empty:
                    weather_df["location_id"] = int(location_id)
                    weather_df["date"] = pd.to_datetime(weather_df["date"])
                    weather_df = weather_df.dropna(subset=['temperature_2m_mean', 'precipitation_sum', 'wind_speed_10m_max'])
                    
                    # Convert to correct types to match schema
                    weather_df["location_id"] = weather_df["location_id"].astype("int32")
                    weather_df["temperature_2m_mean"] = weather_df["temperature_2m_mean"].astype("float64")
                    weather_df["precipitation_sum"] = weather_df["precipitation_sum"].astype("float64")
                    weather_df["wind_speed_10m_max"] = weather_df["wind_speed_10m_max"].astype("float64")
                    weather_df["wind_direction_10m_dominant"] = weather_df["wind_direction_10m_dominant"].astype("float64")
                    
                    weather_fg.insert(weather_df)
                    location_weather_uploaded.add(location_id)
            
            successful_sensors += 1
            print(f"✅ Uploaded sensor {sensor_id} (location {location_id})")
            break  # Success, exit retry loop
            
        except requests.exceptions.Timeout as e:
            if attempt < max_retries - 1:
                wait_time = (attempt + 1) * 5
                print(f"⚠️  Sensor {sensor_id}: Timeout, retrying in {wait_time}s... (attempt {attempt + 1}/{max_retries})")
                time.sleep(wait_time)
            else:
                failed_sensors += 1
                print(f"❌ Sensor {sensor_id}: Failed after {max_retries} timeout attempts")
                break
                
        except requests.exceptions.RequestException as e:
            if attempt < max_retries - 1:
                wait_time = (attempt + 1) * 5
                print(f"⚠️  Sensor {sensor_id}: {type(e).__name__}, retrying in {wait_time}s... (attempt {attempt + 1}/{max_retries})")
                time.sleep(wait_time)
            else:
                failed_sensors += 1
                print(f"❌ Sensor {sensor_id}: Failed after {max_retries} attempts - {type(e).__name__}")
                break
                
        except Exception as e:
            failed_sensors += 1
            print(f"❌ Sensor {sensor_id}: Unexpected error - {type(e).__name__}: {str(e)[:100]}")
            import traceback
            print(f"   {traceback.format_exc()[:300]}")
            break
    
    # Brief pause between sensors
    time.sleep(0.5)

print(f"\n📊 Collection Summary:")
print(f"   ✅ Successful: {successful_sensors}")
print(f"   ⏭️ Skipped (no data): {skipped_sensors}")
print(f"   ❌ Failed: {failed_sensors}")
print(f"   🌤️ Weather locations uploaded: {len(location_weather_uploaded)}")

🔍 Processing 103 sensor locations...
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (24.61s) 
2026-01-09 11:24:35,161 INFO: 	7 expectation(s) included in expectation_suite.
Validation failed.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1279184/fs/1265800/fg/1911228


Uploading Dataframe: 100.00% |██████████| Rows 1/1 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: air_quality_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279184/jobs/named/air_quality_1_offline_fg_materialization/executions
❌ Sensor 191047: Unexpected error - FeatureStoreException: Features are not compatible with Feature Group schema: 
 - temperature_2m_mean (expected type: 'doub
   Traceback (most recent call last):
  File "C:\Users\krist\AppData\Local\Temp\ipykernel_20376\1599027981.py", line 114, in <module>
    weather_fg.insert(weather_df)
  File "c:\Users\krist\Documents\GitHub\pm25-forecast-openmeteo-aqicn\.venv\Lib\site-packages\hsfs\feature_group.py", line 2940, in ins
2026-01-09 11:25:22,242 INFO: 	7 expectation(s) included in expectation_suite.
Validation failed.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1279184/fs/1265800/fg/1911228


Uploading Dataframe: 100.00% |██████████| Rows 1/1 | Elapsed Time: 00:01 | Remaining Time: 00:00


2026-01-09 11:25:31,160 WARNING: UserWarning: Materialization job is already running, aborting new execution.Please wait for the current execution to finish before triggering a new one.You can check the status of the current execution using `fg.materialization_job.get_state()`.or `fg.materialization_job.get_final_state()` or check it out in the Hopsworks UI.at https://c.app.hopsworks.ai:443/p/1279184/jobs/named/air_quality_1_offline_fg_materialization.
Use fg.materialization_job.run(args=-op offline_fg_materialization -path hdfs:///Projects/kristina_titanic/Resources/jobs/air_quality_1_offline_fg_materialization/config_1767871095641) to trigger the materialization job again.

❌ Sensor 59410: Unexpected error - FeatureStoreException: Features are not compatible with Feature Group schema: 
 - temperature_2m_mean (expected type: 'doub
   Traceback (most recent call last):
  File "C:\Users\krist\AppData\Local\Temp\ipykernel_20376\1599027981.py", line 114, in <module>
    weather_fg.insert(

Uploading Dataframe: 100.00% |██████████| Rows 1/1 | Elapsed Time: 00:01 | Remaining Time: 00:00


2026-01-09 11:25:42,732 WARNING: UserWarning: Materialization job is already running, aborting new execution.Please wait for the current execution to finish before triggering a new one.You can check the status of the current execution using `fg.materialization_job.get_state()`.or `fg.materialization_job.get_final_state()` or check it out in the Hopsworks UI.at https://c.app.hopsworks.ai:443/p/1279184/jobs/named/air_quality_1_offline_fg_materialization.
Use fg.materialization_job.run(args=-op offline_fg_materialization -path hdfs:///Projects/kristina_titanic/Resources/jobs/air_quality_1_offline_fg_materialization/config_1767871095641) to trigger the materialization job again.

❌ Sensor 113542: Unexpected error - FeatureStoreException: Features are not compatible with Feature Group schema: 
 - temperature_2m_mean (expected type: 'doub
   Traceback (most recent call last):
  File "C:\Users\krist\AppData\Local\Temp\ipykernel_20376\1599027981.py", line 114, in <module>
    weather_fg.insert

Uploading Dataframe: 0.00% |          | Rows 0/1 | Elapsed Time: 00:00 | Remaining Time: ?

KeyboardInterrupt: 

In [ ]:
# # latest_per_sensor = (
# #     historical_df.groupby("sensor_id")["date"]
# #     .max()
# #     .to_dict()
# # )

# # incremental.run_incremental_update(
# #     sensor_metadata_fg,
# #     air_quality_fg,
# #     weather_fg,
# #     latest_per_sensor,
# #     AQICN_API_KEY
# # )

# aqs = []
# weathers = []
# location_weather_cache = {}  # Cache weather by location_id to avoid duplicate API calls

# successful_sensors = 0
# failed_sensors = 0
# skipped_sensors = 0

# print(f"🔍 Processing {len(metadata_df)} sensor locations...")

# for sensor_id, meta in metadata_df.iterrows():
#     max_retries = 3
    
#     for attempt in range(max_retries):
#         try:
#             aq_today_df, weather_df, location_id = fetchers.fetch_data_for_sensor(sensor_id, meta, today, AQICN_API_KEY)

#             # Format air quality data
#             aq_today_df = aq_today_df.assign(
#                 sensor_id=int(sensor_id),
#                 street=meta["street"],
#                 city=meta["city"],
#                 country=meta["country"],
#                 feed_url=meta["aqicn_url"],
#             )
#             aq_today_df["date"] = pd.to_datetime(aq_today_df["date"])
            
#             # Check if we actually got data
#             if aq_today_df.empty:
#                 print(f"⏭️ Sensor {sensor_id}: No AQ data available")
#                 skipped_sensors += 1
#                 break
            
#             aqs.append(aq_today_df)

#             # Weather is per location_id, not per sensor - cache it
#             if location_id not in location_weather_cache:
#                 weather_df["location_id"] = int(location_id)
#                 weather_df["date"] = pd.to_datetime(weather_df["date"])
#                 location_weather_cache[location_id] = weather_df
#                 weathers.append(weather_df)
            
#             successful_sensors += 1
#             print(f"✅ Processed sensor {sensor_id} (location {location_id})")
#             break  # Success, exit retry loop
            
#         except requests.exceptions.Timeout as e:
#             if attempt < max_retries - 1:
#                 wait_time = (attempt + 1) * 5  # 5s, 10s, 15s
#                 print(f"⚠️  Sensor {sensor_id}: Timeout, retrying in {wait_time}s... (attempt {attempt + 1}/{max_retries})")
#                 time.sleep(wait_time)
#             else:
#                 failed_sensors += 1
#                 print(f"❌ Sensor {sensor_id}: Failed after {max_retries} timeout attempts")
#                 break
                
#         except requests.exceptions.RequestException as e:
#             if attempt < max_retries - 1:
#                 wait_time = (attempt + 1) * 5
#                 print(f"⚠️  Sensor {sensor_id}: {type(e).__name__}, retrying in {wait_time}s... (attempt {attempt + 1}/{max_retries})")
#                 time.sleep(wait_time)
#             else:
#                 failed_sensors += 1
#                 print(f"❌ Sensor {sensor_id}: Failed after {max_retries} attempts - {type(e).__name__}")
#                 break
                
#         except Exception as e:
#             failed_sensors += 1
#             print(f"❌ Sensor {sensor_id}: Unexpected error - {type(e).__name__}: {str(e)[:100]}")
#             break
    
#     # Brief pause between sensors to avoid rate limiting
#     time.sleep(0.5)

# print(f"\n📊 Collection Summary:")
# print(f"   ✅ Successful: {successful_sensors}")
# print(f"   ⏭️ Skipped (no data): {skipped_sensors}")
# print(f"   ❌ Failed: {failed_sensors}")
# print(f"   🌤️ Unique locations: {len(location_weather_cache)}")

🔍 Processing 103 sensor locations...
✅ Processed sensor 191047 (location 15)
✅ Processed sensor 59410 (location 49)
✅ Processed sensor 113542 (location 6)
✅ Processed sensor 59497 (location 50)
✅ Processed sensor 420664 (location 33)
✅ Processed sensor 60076 (location 58)
✅ Processed sensor 61714 (location 68)
✅ Processed sensor 198559 (location 19)
✅ Processed sensor 60535 (location 59)
✅ Processed sensor 81505 (location 94)
✅ Processed sensor 57421 (location 42)
✅ Processed sensor 59893 (location 55)
✅ Processed sensor 89584 (location 100)
✅ Processed sensor 409513 (location 30)
✅ Processed sensor 407335 (location 29)
✅ Processed sensor 194215 (location 17)
✅ Processed sensor 105325 (location 1)
✅ Processed sensor 77488 (location 88)
✅ Processed sensor 63646 (location 75)
✅ Processed sensor 149242 (location 11)
✅ Processed sensor 58912 (location 45)
✅ Processed sensor 80773 (location 93)
✅ Processed sensor 113539 (location 5)
✅ Processed sensor 84085 (location 97)
✅ Processed sensor 

KeyboardInterrupt: 

In [ ]:
aq_df = pd.concat(aqs)
aq_df["pm25"] = pd.to_numeric(aq_df["pm25"], errors="coerce").astype("float64")
aq_df["date"] = pd.to_datetime(aq_df["date"]).dt.tz_localize(None)
aq_df = aq_df.drop(columns=["url"], errors="ignore")

In [ ]:
aq_df = pd.concat(aqs) if aqs else pd.DataFrame()
if not aq_df.empty:
    aq_df["pm25"] = pd.to_numeric(aq_df["pm25"], errors="coerce").astype("float64")
    aq_df["date"] = pd.to_datetime(aq_df["date"]).dt.tz_localize(None)
    aq_df = aq_df.drop(columns=["url"], errors="ignore")

    # Data quality check 1: Remove rows with missing PM2.5 values
    initial_count = len(aq_df)
    aq_df = aq_df.dropna(subset=['pm25'])
    if len(aq_df) < initial_count:
        print(f"🧹 Removed {initial_count - len(aq_df)} rows with missing PM2.5 values")

# Get historical data for rolling window and lagged features
historical_start = today - datetime.timedelta(days=4)
historical_df = pd.DataFrame()

# Read historical data from feature group and filter for the last 4 days
try:
    cols = [f.name for f in air_quality_fg.features] 
    historical_df = air_quality_fg.read(cols)
    if not historical_df.empty:
        historical_df["date"] = pd.to_datetime(historical_df["date"]).dt.tz_localize(None)
        historical_df = historical_df[
            (historical_df["date"] >= historical_start) & (historical_df["date"] < today)
        ][["date", "sensor_id", "pm25"]]
except Exception as e:
    print(f"Error reading historical data: {e}")
    # pass

In [ ]:
combined_df = pd.concat([historical_df, aq_df], ignore_index=True) if not historical_df.empty else aq_df
if not combined_df.empty:
    combined_df = feature_engineering.add_rolling_window_feature(combined_df, window_days=3, column="pm25", new_column="pm25_rolling_3d")
    combined_df = feature_engineering.add_lagged_features(combined_df, column="pm25", lags=[1, 2, 3])
    combined_df = feature_engineering.add_nearby_sensor_feature(combined_df, metadata_df.to_dict('index'), column="pm25_lag_1d", n_closest=3)
    
    # Data quality check 2: Clean up NaNs created by feature engineering
    before_cleaning = len(combined_df[combined_df["date"].dt.date == today])
    
    # Only keep today's data and remove rows where essential features are NaN
    aq_df = combined_df[combined_df["date"].dt.date == today].copy()
    
    # Remove rows where pm25 is still NaN after all processing
    aq_df = aq_df.dropna(subset=['pm25'])
    
    after_cleaning = len(aq_df)
    if before_cleaning > after_cleaning:
        print(f"🧹 Removed {before_cleaning - after_cleaning} rows with NaN values after feature engineering")
    
    print(f"📊 Final data quality: {len(aq_df)} clean rows ready for feature store")
else:
    aq_df = pd.DataFrame()
    print("⚠️  No data available for processing")
aq_df.head()

In [ ]:
weather_df = pd.concat(weathers) if weathers else pd.DataFrame()
if not weather_df.empty:
    weather_df["date"] = pd.to_datetime(weather_df["date"])
    
    # Data quality check 3: Remove rows with missing weather data
    initial_weather_count = len(weather_df)
    weather_df = weather_df.dropna(subset=['temperature_2m_mean', 'precipitation_sum', 'wind_speed_10m_max'])
    
    # Convert to float32 to match Hopsworks feature group schema
    weather_df["temperature_2m_mean"] = weather_df["temperature_2m_mean"].astype("float32")
    weather_df["precipitation_sum"] = weather_df["precipitation_sum"].astype("float32")
    weather_df["wind_speed_10m_max"] = weather_df["wind_speed_10m_max"].astype("float32")
    weather_df["wind_direction_10m_dominant"] = weather_df["wind_direction_10m_dominant"].astype("float32")
    
    if len(weather_df) < initial_weather_count:
        print(f"🧹 Removed {initial_weather_count - len(weather_df)} rows with missing weather data")
    
    print(f"🌤️  Weather data quality: {len(weather_df)} clean weather rows")
else:
    print("⚠️  No weather data available")
weather_df.head()

In [ ]:
print(weather_df['date'].unique())

In [ ]:
# Final validation before inserting to feature store
if not aq_df.empty and not weather_df.empty:
    print(f"✅ Inserting {len(aq_df)} air quality rows and {len(weather_df)} weather rows to feature store")
    air_quality_fg.insert(aq_df)
    weather_fg.insert(weather_df)
    print("📁 Data successfully inserted to feature store")
else:
    if aq_df.empty:
        print("⚠️  No clean air quality data to insert")
    if weather_df.empty:
        print("⚠️  No clean weather data to insert")

In [ ]:
print(weather_df['date'].unique())

In [ ]:
print(weather_df.dtypes)

In [ ]:
print(aq_df.dtypes)

In [ ]:
print(air_quality_fg.read().dtypes)